In [63]:
# Import all necessary libraries and setup the environment for matplotlib
%matplotlib inline

from pyspark.sql import SparkSession

from pyspark.ml.feature import PCA
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
import numpy as np
import matplotlib.pyplot as plt

In [16]:
spark = SparkSession \
    .builder \
    .appName("Pyspark Assignment2 Comp5349") \
    .getOrCreate()

pc_rev = "s3://amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"

pc_df = spark.read.csv(pc_rev,header=True,sep='\t')

In [17]:
pc_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [20]:
pc_df.count()

4751577

In [21]:
pc_df.select("customer_id").distinct().count()

1940732

In [22]:
pc_df.select("product_id").distinct().count()

782326

In [37]:
pc_df.groupBy("customer_id").count().orderBy("count", ascending=False).show(10)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   50736950| 7168|
|   38214553| 5412|
|   51184997| 5369|
|   18116317| 4222|
|   23267387| 4023|
|   50345651| 3793|
|   14539589| 2896|
|   15725862| 2842|
|   19380211| 2592|
|   20018062| 2568|
+-----------+-----+
only showing top 10 rows



In [48]:
customer_median = pc_df.groupBy("customer_id").count().orderBy("count", ascending=False).approxQuantile("count",[0.5],0.25)

In [38]:
pc_df.groupBy("product_id").count().orderBy("count", ascending=False).show(10)

+----------+-----+
|product_id|count|
+----------+-----+
|B00008OWZG| 3936|
|B0000AGWEC| 3326|
|B00MIA0KGY| 2699|
|B00NEJ7MMI| 2420|
|B000089RVX| 2376|
|B004EBT5CU| 2106|
|B0026P3G12| 2080|
|B00009PRZF| 2026|
|B00004XONN| 1901|
|B00006J6VG| 1793|
+----------+-----+
only showing top 10 rows



In [49]:
product_median = pc_df.groupBy("product_id").count().orderBy("count", ascending=False).approxQuantile("count",[0.5],0.25)

In [52]:
customer_median[0]

2457.0

In [53]:
product_median[0]

673.0

In [87]:
product_filter = pc_df.groupBy("product_id").count().filter("count<673")

In [88]:
product_filter.orderBy("count", ascending = False).show(5)

+----------+-----+
|product_id|count|
+----------+-----+
|B00001NFCY|  672|
|B0001MBG0U|  670|
|B000042OI4|  668|
|B000EMGAOY|  668|
|B008NW67E0|  667|
+----------+-----+
only showing top 5 rows



In [89]:
from pyspark.sql.functions import col

pc_df_filter1 = pc_df.join(product_filter, "product_id","inner")

In [90]:
pc_df_filter1.count()

4587256